In [3]:
import numpy as np
import torch
import open3d as o3d
from lidar_utils.lidar_utils import (read_point_cloud, read_config, filter_point_cloud, visualize_pc, 
                                    point_cloud_to_tensor, voxelize, read_label, label_2_bb, 
                                    read_calib_file, visualize_bb, visualize_bb_from_file)

idx = 502
str_idx = f"{idx:06}"
config_file = r'E:\Deep Learning Projects\3d_object_detection\voxel_net\config\train_config.yaml'
training_file_path = rf"E:\Deep Learning Projects\datasets\training\velodyne\{str_idx}.bin"
training_label_path = rf"E:\Deep Learning Projects\datasets\training\label_2\{str_idx}.txt"
training_calib_path = rf"E:\Deep Learning Projects\datasets\training\calib\{str_idx}.txt"

point_cloud = read_point_cloud(training_file_path)
cfg = read_config(config_file)
point_cloud = filter_point_cloud(cfg, point_cloud)

voxel_coords = voxelize(point_cloud, cfg)

x_min, x_max = cfg['x_min'], cfg['x_max']
y_min, y_max = cfg['y_min'], cfg['y_max']
z_min, z_max = cfg['z_min'], cfg['z_max'] 
vd, vh, vw = cfg['vd'], cfg['vh'], cfg['vw']
T = cfg['T']

unique_voxel_coords, inverse_index = np.unique(voxel_coords, axis=0, return_inverse=True)
K = unique_voxel_coords.shape[0]
tensor = np.zeros((K, T, 7))
for i in range(len(unique_voxel_coords)):
    indices = np.flatnonzero(inverse_index==i)
    # print(unique_voxel_coords[i])
    row = point_cloud[indices][:T, :]
    aug = row[:,:3] - row[:,:3].mean(0)
    new_row = np.hstack((row, aug))
    tensor[i, :min(T, len(indices)), :] = new_row


visualize_bb_from_file(training_file_path, training_label_path, training_calib_path, config_file)

In [ ]:
# _, h, w, l, x, y, z, theta = i
read_label(training_label_path)

[['Car', 1.6, 1.76, 3.84, -0.37, 1.26, 32.14, -1.58],
 ['Car', 1.4, 1.64, 3.8, -8.44, 1.56, 9.49, -1.57],
 ['Car', 1.65, 1.66, 3.96, -7.86, 1.47, 15.59, -1.57],
 ['Car', 1.48, 1.64, 3.81, -3.65, 1.49, 14.49, -1.56],
 ['Car', 1.44, 1.71, 3.78, -7.58, 1.4, 22.42, -1.57],
 ['Car', 1.37, 1.61, 4.21, -3.4, 1.32, 26.88, -1.55],
 ['Car', 1.45, 1.64, 3.86, -7.73, 1.18, 41.04, -1.57],
 ['Van', 2.74, 2.09, 5.96, -4.81, 0.99, 59.1, -1.6],
 ['Car', 1.46, 1.51, 3.83, -12.13, 1.48, 17.34, -1.57]]

In [ ]:
def pc_to_tensor(cfg, pc):
    x_min = cfg['x_min'] 
    y_min = cfg['y_min'] 
    z_min = cfg['z_min']
    vd, vh, vw = cfg['vd'], cfg['vh'], cfg['vw']
    T = cfg['T']

    points, intensities = pc[:, :3], pc[:, 3]
    voxel_coords = ((points - np.array([x_min, y_min, z_min]))/np.array([vw, vh, vd])).astype(np.int32)
    K = len(np.unique(voxel_coords, axis=0))
    tensor = np.zeros((K, T, 7))

    unique_voxels, inverse_indices = np.unique(voxel_coords, return_inverse=True, axis=0)

    for i in range(len(unique_voxels)):
        indices = inverse_indices==i
        points = pc[indices]
        centroid = np.mean(points[:, :3], axis=0)
        augment = points[:, :3] - centroid
        points = np.hstack((points, augment))
        limit = min(len(points), T)
        points = points[:limit, :]
        tensor[i, :limit, :] = points

    return torch.from_numpy(tensor)

array = pc_to_tensor(cfg, pc)



tensor([[    0,     0,     0,  ..., 10141, 10141, 10141],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     1,     2,  ...,     0,     1,     2]])